In [1]:
import sqlite3
import json
from operator import itemgetter
import pandas as pd
from zipfile import ZipFile

In [2]:
con = sqlite3.connect("dmi.db")
cur = con.cursor()

In [3]:
cur.execute(
    '''CREATE TABLE IF NOT EXISTS feature (
        stationId TEXT,
        created_dt TEXT,
        observed_dt TEXT,
        parameterId TEXT,
        value FLOAT,
        id TEXT
        )
    ''')

cur.execute(
    '''CREATE TABLE IF NOT EXISTS stations (
        stationId TEXT,
        name TEXT,
        country TEXT,
        owner TEXT,
        type TEXT,
        status TEXT,
        stationHeight FLOAT,
        latitude FLOAT,
        longitude FLOAT,
        region int,
        created_dt TEXT,
        operationFrom_dt TEXT,
        operationTo_dt TEXT,
        updated_dt TEXT,
        validFrom_dt TEXT,
        validTo_dt TEXT,
        wmoCountryCode INT,
        wmoStationId TEXT
        )
    ''')

Metrologisk observationsstationer hentet fra: https://confluence.govcloud.dk/display/FDAPI/Meteorological+Observation+Data+Stations

In [ ]:
maalestationer = pd.read_csv('målestationer.csv')

all.zip filen som indeholder alle observationerne er hentet her fra https://confluence.govcloud.dk/display/FDAPI/Download

In [ ]:
%%time
# tager ca. at køre

n_filer_behandlet = 0
with ZipFile('all.zip', 'r') as all_data:
    data_files = all_data.namelist()
    for df in data_files:
        d_table = []
        for l in all_data.open(df):
            d = json.loads(l)
            if d['type'] == 'Feature':
                d_prop = d['properties']
                row = [d_prop.get('stationId', None),
                       d_prop.get('created', None),
                       d_prop.get('observed', None),
                       d_prop.get('parameterId', None),
                       d_prop.get('value', None),
                       d.get('id', None)]
                d_table.append(row)
        cur.executemany('insert into feature values (?, ?, ?, ?, ?, ?);', d_table)

        n_filer_behandlet += 1
        if n_filer_behandlet % 2500 == 0:
            print(f'{n_filer_behandlet/len(data_files):.0%} filer behandlet')

In [ ]:
pd.read_sql('select count(*) as antal_observationer from feature', con, parse_dates=['created_dt', 'observed_dt'])